<a href="https://colab.research.google.com/github/ChyYasir/BioSignal_Detection/blob/main/auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Configure Git (only once per session)
!git config --global user.email "chyyasir2000@gmail.com"
!git config --global user.name "ChyYasir"


In [ ]:
!git clone https://github.com/ChyYasir/BioSignal_Detection.git


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Softmax
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate
import tensorflow.keras.backend as K
from imblearn.over_sampling import SMOTE


In [ ]:
# @title
import pandas as pd

cesarean_df = pd.read_csv('/content/drive/MyDrive/ML/early_cesarean_features.csv')
induced_cesarean_df = pd.read_csv('/content/drive/MyDrive/ML/early_induced-cesarean_features.csv')
spontaneous_df = pd.read_csv('/content/drive/MyDrive/ML/early_spontaneous_features.csv')

cesarean_df['label'] = 0
induced_cesarean_df['label'] = 1
spontaneous_df['label'] = 2

merged_df = pd.concat([cesarean_df, induced_cesarean_df, spontaneous_df])

merged_df.to_csv('/content/drive/MyDrive/ML/features.csv', index=False)

In [2]:
data = pd.read_csv("/content/drive/MyDrive/ML/features.csv")

X = data.drop("label", axis=1).values
y = data["label"].values



In [3]:
# Check class distribution
class_counts = np.unique(y, return_counts=True)
print("Class distribution before balancing:")
for label, count in zip(class_counts[0], class_counts[1]):
    print(f"Class {label}: {count} samples")

# Balance data using SMOTE if imbalanced
if len(np.unique(class_counts[1])) > 1:  # Check if counts differ
    smote = SMOTE(random_state=42)
    X_balanced, y_balanced = smote.fit_resample(X, y)
    print("\nClass distribution after SMOTE balancing:")
    class_counts_balanced = np.unique(y_balanced, return_counts=True)
    for label, count in zip(class_counts_balanced[0], class_counts_balanced[1]):
        print(f"Class {label}: {count} samples")
else:
    print("\nData is already balanced.")
    X_balanced, y_balanced = X, y


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_balanced)

num_classes = len(np.unique(y_balanced))
y_categorical = to_categorical(y_balanced, num_classes=num_classes)

# Verify data
print(f"\nX_scaled shape: {X_scaled.shape}")
print(f"y_categorical shape: {y_categorical.shape}")
print(f"Number of features: {X_scaled.shape[1]}")

Class distribution before balancing:
Class 0: 17 samples
Class 1: 19 samples
Class 2: 216 samples

Class distribution after SMOTE balancing:
Class 0: 216 samples
Class 1: 216 samples
Class 2: 216 samples

X_scaled shape: (648, 16)
y_categorical shape: (648, 3)
Number of features: 16


In [4]:
# Plot confusion matrix
def plot_confusion_matrix(y_true_all, y_pred_all, classes, save_path='confusion_matrix.png'):
    y_true_flat = np.concatenate([np.argmax(y_true, axis=1) for y_true in y_true_all])
    y_pred_flat = np.concatenate([np.argmax(y_pred, axis=1) for y_pred in y_pred_all])

    cm = confusion_matrix(y_true_flat, y_pred_flat)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(save_path)
    plt.show()
    plt.close()

In [5]:
# Plot ROC curve (multi-class, one-vs-rest)
def plot_roc_curve(y_true_all, y_score_all, num_classes, class_names, save_path='roc_curve.png'):
    mean_fpr = np.linspace(0, 1, 100)
    plt.figure(figsize=(8, 6))

    mean_tprs = []
    aucs = []

    for c in range(num_classes):
        tprs = []
        class_aucs = []
        for i in range(len(y_true_all)):
            fpr, tpr, _ = roc_curve(y_true_all[i][:, c], y_score_all[i][:, c])
            class_aucs.append(auc(fpr, tpr))
            tpr_interp = np.interp(mean_fpr, fpr, tpr)
            tpr_interp[0] = 0.0
            tprs.append(tpr_interp)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(class_aucs)
        aucs.append(mean_auc)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

        plt.plot(mean_fpr, mean_tpr, lw=2, alpha=0.8,
                 label=f'{class_names[c]} (AUC = {mean_auc:.2f} ± {std_auc:.2f})')
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, alpha=0.2)

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves (One-vs-Rest)')
    plt.legend(loc='lower right')
    plt.savefig(save_path)
    plt.show()
    plt.close()
    return np.mean(aucs)

# **One Layer**

In [ ]:
def build_ssae_model(input_dim, num_classes):
    # First and only autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu', activity_regularizer=l1(10e-5))(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    print(autoencoder.summary())
    print(encoder.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer = encoder(input_layer)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder, encoder

# Function to train the SSAE model with one autoencoder
def train_ssae_model(X_train, autoencoder):
    # Train the first and only autoencoder with matching input and output dimensions
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the model
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []

    y_true_all = []
    y_score_all = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder, encoder = build_ssae_model(X_train.shape[1], num_classes)

            # train_ssae_model(X_train, encoder)
            train_ssae_model(X_train, autoencoder)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))
            y_true_all.append(y_test)
            y_score_all.append(y_pred)

    accuracy = np.mean(accuracy_list)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f1 = np.mean(f1_list)

    # Plot visualizations
    class_names = ['Cesarean', 'Induced-C', 'Spontaneous']
    mean_auc = plot_roc_curve(y_true_all, y_score_all, num_classes, class_names, save_path='roc_curve.png')
    plot_confusion_matrix(y_true_all, y_score_all, classes=class_names, save_path='confusion_matrix.png')

    return accuracy, precision, recall, f1

# Evaluate the model using 10-fold cross-validation with 30 repetitions
accuracy, precision, recall, f1 = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# **Two Layer**

In [ ]:


# Function to build SSAE model with two encoders
def build_ssae_model(input_dim, num_classes):
    # First autoencoder
    input_layer = Input(shape=(input_dim,))
    # encoded1 = Dense(64, activation='relu', activity_regularizer=l1(10e-5))(input_layer)
    # decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)
    # encoded1 = Dense(18, activation='relu', name='encoded1', kernel_regularizer='l2')(input_layer)
    # decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)
    encoded1 = Dense(18, activation='relu', name='encoded1', activity_regularizer=l1(1e-4))(input_layer)
    decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)

    autoencoder1 = Model(input_layer, decoded1)
    encoder1 = Model(input_layer, encoded1)

    print(autoencoder1.summary())
    print(encoder1.summary())

    # Second autoencoder
    # input_encoded1 = Input(shape=(64,))
    # encoded2 = Dense(32, activation='relu', activity_regularizer=l1(10e-5))(input_encoded1)
    # decoded2 = Dense(64, activation='sigmoid')(encoded2)

    # input_encoded1 = Input(shape=(18,))
    # encoded2 = Dense(12, activation='relu', name='encoded2', kernel_regularizer='l2')(input_encoded1)
    # decoded2 = Dense(18, activation='sigmoid')(encoded2)

    input_encoded1 = Input(shape=(18,))
    encoded2 = Dense(12, activation='relu', name='encoded2', activity_regularizer=l1(1e-4))(input_encoded1)
    decoded2 = Dense(18, activation='sigmoid')(encoded2)

    autoencoder2 = Model(input_encoded1, decoded2)
    encoder2 = Model(input_encoded1, encoded2)

    print(autoencoder2.summary())
    print(encoder2.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer1 = encoder1(input_layer)
    encoded_layer2 = encoder2(encoded_layer1)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer2)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder1, encoder1, autoencoder2, encoder2

# Function to train SSAE model with two encoders
def train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2):
    # Train the first autoencoder
    autoencoder1.compile(optimizer='adam', loss='mse')
    autoencoder1.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the input data using the first encoder
    X_encoded1 = encoder1.predict(X_train)

    # Train the second autoencoder using the encoded data from the first encoder
    autoencoder2.compile(optimizer='adam', loss='mse')
    autoencoder2.fit(X_encoded1, X_encoded1, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the SSAE model with two encoders
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    y_true_all = []
    y_score_all = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder1, encoder1, autoencoder2, encoder2 = build_ssae_model(X_train.shape[1], num_classes)

            # Train the SSAE model
            train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))

            y_true_all.append(y_test)
            y_score_all.append(y_pred)

    accuracy = np.mean(accuracy_list)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)

    f1 = np.mean(f1_list)
    # Plot visualizations
    class_names = ['Cesarean', 'Induced-C', 'Spontaneous']
    mean_auc = plot_roc_curve(y_true_all, y_score_all, num_classes, class_names, save_path='roc_curve.png')
    plot_confusion_matrix(y_true_all, y_score_all, classes=class_names, save_path='confusion_matrix.png')

    return accuracy, precision, recall, f1

# Assuming X_scaled and y_categorical are already defined and preprocessed
num_classes = y_categorical.shape[1]

# Evaluate the model using 10-fold cross-validation with 30 repetitions
accuracy, precision, recall, f1 = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


# **Three Layer**


In [ ]:
# Function to build SSAE model with three encoders
def build_ssae_model(input_dim, num_classes):
    # First autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded1 = Dense(18, activation='relu', name='encoded1', activity_regularizer=l1(1e-4))(input_layer)
    decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)

    autoencoder1 = Model(input_layer, decoded1)
    encoder1 = Model(input_layer, encoded1)

    print("Autoencoder 1 summary:")
    print(autoencoder1.summary())
    print("Encoder 1 summary:")
    print(encoder1.summary())

    # Second autoencoder
    input_encoded1 = Input(shape=(18,))
    encoded2 = Dense(12, activation='relu', name='encoded2', activity_regularizer=l1(1e-4))(input_encoded1)
    decoded2 = Dense(18, activation='sigmoid')(encoded2)

    autoencoder2 = Model(input_encoded1, decoded2)
    encoder2 = Model(input_encoded1, encoded2)

    print("Autoencoder 2 summary:")
    print(autoencoder2.summary())
    print("Encoder 2 summary:")
    print(encoder2.summary())

    # Third autoencoder
    input_encoded2 = Input(shape=(12,))
    encoded3 = Dense(8, activation='relu', name='encoded3', activity_regularizer=l1(1e-4))(input_encoded2)
    decoded3 = Dense(12, activation='sigmoid')(encoded3)

    autoencoder3 = Model(input_encoded2, decoded3)
    encoder3 = Model(input_encoded2, encoded3)

    print("Autoencoder 3 summary:")
    print(autoencoder3.summary())
    print("Encoder 3 summary:")
    print(encoder3.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer1 = encoder1(input_layer)
    encoded_layer2 = encoder2(encoded_layer1)
    encoded_layer3 = encoder3(encoded_layer2)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer3)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3

# Function to train SSAE model with three encoders
def train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3):
    # Train the first autoencoder
    autoencoder1.compile(optimizer='adam', loss='mse')
    autoencoder1.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the input data using the first encoder
    X_encoded1 = encoder1.predict(X_train, verbose=0)

    # Train the second autoencoder
    autoencoder2.compile(optimizer='adam', loss='mse')
    autoencoder2.fit(X_encoded1, X_encoded1, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the data using the second encoder
    X_encoded2 = encoder2.predict(X_encoded1, verbose=0)

    # Train the third autoencoder
    autoencoder3.compile(optimizer='adam', loss='mse')
    autoencoder3.fit(X_encoded2, X_encoded2, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the SSAE model with three encoders
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    y_true_all = []
    y_score_all = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3 = build_ssae_model(X_train.shape[1], num_classes)

            # Train the SSAE model
            train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test, verbose=0)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))

            y_true_all.append(y_test)
            y_score_all.append(y_pred)

    accuracy = np.mean(accuracy_list) * 100
    precision = np.mean(precision_list) * 100
    recall = np.mean(recall_list) * 100
    f1 = np.mean(f1_list)

    # Plot visualizations using functions defined elsewhere
    class_names = ['Cesarean', 'Induced-C', 'Spontaneous']
    mean_auc = plot_roc_curve(y_true_all, y_score_all, num_classes, class_names, save_path='roc_curve.png')
    plot_confusion_matrix(y_true_all, y_score_all, classes=class_names, save_path='confusion_matrix.png')

    return accuracy, precision, recall, f1, mean_auc

# Evaluate the model
accuracy, precision, recall, f1, mean_auc = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1-score: {f1:.4f}")
print(f"Mean AUC: {mean_auc:.4f}")

# **Four Layer**


In [ ]:
# Function to build SSAE model with four encoders
def build_ssae_model(input_dim, num_classes):
    # First autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded1 = Dense(18, activation='relu', name='encoded1', activity_regularizer=l1(1e-4))(input_layer)
    decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)

    autoencoder1 = Model(input_layer, decoded1)
    encoder1 = Model(input_layer, encoded1)

    print("Autoencoder 1 summary:")
    print(autoencoder1.summary())
    print("Encoder 1 summary:")
    print(encoder1.summary())

    # Second autoencoder
    input_encoded1 = Input(shape=(18,))
    encoded2 = Dense(12, activation='relu', name='encoded2', activity_regularizer=l1(1e-4))(input_encoded1)
    decoded2 = Dense(18, activation='sigmoid')(encoded2)

    autoencoder2 = Model(input_encoded1, decoded2)
    encoder2 = Model(input_encoded1, encoded2)

    print("Autoencoder 2 summary:")
    print(autoencoder2.summary())
    print("Encoder 2 summary:")
    print(encoder2.summary())

    # Third autoencoder
    input_encoded2 = Input(shape=(12,))
    encoded3 = Dense(8, activation='relu', name='encoded3', activity_regularizer=l1(1e-4))(input_encoded2)
    decoded3 = Dense(12, activation='sigmoid')(encoded3)

    autoencoder3 = Model(input_encoded2, decoded3)
    encoder3 = Model(input_encoded2, encoded3)

    print("Autoencoder 3 summary:")
    print(autoencoder3.summary())
    print("Encoder 3 summary:")
    print(encoder3.summary())

    # Fourth autoencoder
    input_encoded3 = Input(shape=(8,))
    encoded4 = Dense(6, activation='relu', name='encoded4', activity_regularizer=l1(1e-4))(input_encoded3)
    decoded4 = Dense(8, activation='sigmoid')(encoded4)

    autoencoder4 = Model(input_encoded3, decoded4)
    encoder4 = Model(input_encoded3, encoded4)

    print("Autoencoder 4 summary:")
    print(autoencoder4.summary())
    print("Encoder 4 summary:")
    print(encoder4.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer1 = encoder1(input_layer)
    encoded_layer2 = encoder2(encoded_layer1)
    encoded_layer3 = encoder3(encoded_layer2)
    encoded_layer4 = encoder4(encoded_layer3)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer4)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3, autoencoder4, encoder4

# Function to train SSAE model with four encoders
def train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3, autoencoder4, encoder4):
    # Train the first autoencoder
    autoencoder1.compile(optimizer='adam', loss='mse')
    autoencoder1.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the input data using the first encoder
    X_encoded1 = encoder1.predict(X_train, verbose=0)

    # Train the second autoencoder
    autoencoder2.compile(optimizer='adam', loss='mse')
    autoencoder2.fit(X_encoded1, X_encoded1, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the data using the second encoder
    X_encoded2 = encoder2.predict(X_encoded1, verbose=0)

    # Train the third autoencoder
    autoencoder3.compile(optimizer='adam', loss='mse')
    autoencoder3.fit(X_encoded2, X_encoded2, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the data using the third encoder
    X_encoded3 = encoder3.predict(X_encoded2, verbose=0)

    # Train the fourth autoencoder
    autoencoder4.compile(optimizer='adam', loss='mse')
    autoencoder4.fit(X_encoded3, X_encoded3, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the SSAE model with four encoders
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    y_true_all = []
    y_score_all = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3, autoencoder4, encoder4 = build_ssae_model(X_train.shape[1], num_classes)

            # Train the SSAE model
            train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2, encoder2, autoencoder3, encoder3, autoencoder4, encoder4)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test, verbose=0)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))

            y_true_all.append(y_test)
            y_score_all.append(y_pred)

    accuracy = np.mean(accuracy_list) * 100
    precision = np.mean(precision_list) * 100
    recall = np.mean(recall_list) * 100
    f1 = np.mean(f1_list)

    # Plot visualizations using functions defined elsewhere
    class_names = ['Cesarean', 'Induced-C', 'Spontaneous']
    mean_auc = plot_roc_curve(y_true_all, y_score_all, num_classes, class_names, save_path='roc_curve.png')
    plot_confusion_matrix(y_true_all, y_score_all, classes=class_names, save_path='confusion_matrix.png')

    return accuracy, precision, recall, f1, mean_auc

# Evaluate the model
accuracy, precision, recall, f1, mean_auc = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1-score: {f1:.4f}")
print(f"Mean AUC: {mean_auc:.4f}")

Autoencoder 1 summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded1 (Dense)                │ (None, 18)             │           306 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           304 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 610 (2.38 KB)

 Trainable params: 610 (2.38 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_171 (InputLayer)    │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoded4 (Dense)                │ (None, 6)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54 (216.00 B)

 Trainable params: 54 (216.00 B)

 Non-trainable params: 0 (0.00 B)

None
